# CAFA-6 Protein Function Prediction - Multi-Aspect MLP Model

**Improvements over baseline:**
- **Multi-aspect heads**: Separate MLP models for C (Cellular Component), F (Molecular Function), P (Biological Process)
- **MLP architecture**: Hidden layer (512 dims) with ReLU activation for non-linear learning
- **Focal Loss**: Handles extreme class imbalance by focusing on hard examples
- **Fmax evaluation**: Local validation matching Kaggle's evaluation

**Architecture**: 3 separate MLP classifiers (1280 → 512 → GO terms) with Focal Loss


In [23]:
# Imports
import numpy as np
import pandas as pd
import os
import scipy.sparse as sp
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import gc

print("✅ All imports loaded")


✅ All imports loaded


In [24]:
# Helper function for paths (works locally and on Kaggle)
import os
def get_data_path(filename):
    if os.path.exists('/kaggle/input'):
        base_path = '/kaggle/input/cafa-6-protein-function-prediction'
    else:
        base_path = '.'
    return os.path.join(base_path, filename)


In [25]:
# Step 1: Load ESM2 Embeddings
print("=== Loading ESM2 Embeddings ===")

def find_embedding_dir(possible_names):
    if os.path.exists('/kaggle/input'):
        for name in possible_names:
            kaggle_path = f'/kaggle/input/{name}'
            if os.path.exists(kaggle_path):
                return kaggle_path
    else:
        for name in possible_names:
            if os.path.exists(name):
                return name
    return None

esm2_possible_names = [
    'cafa-5-ems-2-embeddings-numpy',
    'EMS2_Embeddings',
    'ems2-embeddings'
]

esm2_dir = find_embedding_dir(esm2_possible_names)

if esm2_dir:
    print(f"📁 Found ESM2 embeddings at: {esm2_dir}")
    esm2_train_emb = np.load(f'{esm2_dir}/train_embeddings.npy')
    esm2_train_ids = np.load(f'{esm2_dir}/train_ids.npy')
    esm2_test_emb = np.load(f'{esm2_dir}/test_embeddings.npy')
    esm2_test_ids = np.load(f'{esm2_dir}/test_ids.npy')
    
    print(f"✅ ESM2 Train: {esm2_train_emb.shape} ({len(esm2_train_ids)} proteins)")
    print(f"✅ ESM2 Test: {esm2_test_emb.shape} ({len(esm2_test_ids)} proteins)")
    print(f"   Embedding dimension: {esm2_train_emb.shape[1]}")
else:
    raise ValueError(f"❌ ESM2 embeddings not found! Tried: {esm2_possible_names}")


=== Loading ESM2 Embeddings ===
📁 Found ESM2 embeddings at: EMS2_Embeddings
✅ ESM2 Train: (142246, 1280) (142246 proteins)
✅ ESM2 Test: (141864, 1280) (141864 proteins)
   Embedding dimension: 1280


In [26]:
# Step 2: Load training labels and split by aspect (C/F/P)
print("=== Loading Training Labels & Splitting by Aspect ===")

train_terms = pd.read_csv(get_data_path('Train/train_terms.tsv'), sep='\t')
print(f"Training labels: {len(train_terms)} protein-GO term pairs")
print(f"Unique proteins: {train_terms['EntryID'].nunique()}")
print(f"Unique GO terms: {train_terms['term'].nunique()}")

# Find proteins in both embeddings and labels
train_protein_set = set(train_terms['EntryID'].unique())
embedding_protein_set = set(esm2_train_ids)
common_proteins = list(train_protein_set & embedding_protein_set)

print(f"\nProteins in both: {len(common_proteins)}")

# Filter to common proteins
train_terms_filtered = train_terms[train_terms['EntryID'].isin(common_proteins)].copy()

# Create aspect mapping
go_term_to_aspect = dict(zip(train_terms_filtered['term'], train_terms_filtered['aspect']))

# Split GO terms by aspect
go_terms_C = sorted([term for term in train_terms_filtered['term'].unique() 
                     if go_term_to_aspect[term] == 'C'])
go_terms_F = sorted([term for term in train_terms_filtered['term'].unique() 
                     if go_term_to_aspect[term] == 'F'])
go_terms_P = sorted([term for term in train_terms_filtered['term'].unique() 
                     if go_term_to_aspect[term] == 'P'])

print(f"\nGO terms by aspect (before filtering):")
print(f"  C (Cellular Component): {len(go_terms_C)}")
print(f"  F (Molecular Function): {len(go_terms_F)}")
print(f"  P (Biological Process): {len(go_terms_P)}")
print(f"  Total: {len(go_terms_C) + len(go_terms_F) + len(go_terms_P)}")

# Step 2.5: Filter to allowed terms (IA.tsv) - only terms that are actually evaluated
print("\n=== Filtering to Allowed Terms (IA.tsv) ===")
ia_df = pd.read_csv(get_data_path('IA.tsv'), sep='\t', header=None, names=['term', 'ia_score'])
allowed_terms = set(ia_df['term'].unique())
print(f"Allowed GO terms in IA.tsv: {len(allowed_terms)}")

# Filter each aspect to only allowed terms
go_terms_C = [term for term in go_terms_C if term in allowed_terms]
go_terms_F = [term for term in go_terms_F if term in allowed_terms]
go_terms_P = [term for term in go_terms_P if term in allowed_terms]

print(f"\nGO terms by aspect (after filtering to allowed terms):")
print(f"  C (Cellular Component): {len(go_terms_C)}")
print(f"  F (Molecular Function): {len(go_terms_F)}")
print(f"  P (Biological Process): {len(go_terms_P)}")
print(f"  Total: {len(go_terms_C) + len(go_terms_F) + len(go_terms_P)}")
print(f"\n✅ Filtered to allowed terms - model will focus on terms that are actually scored!")


=== Loading Training Labels & Splitting by Aspect ===
Training labels: 537027 protein-GO term pairs
Unique proteins: 82404
Unique GO terms: 26125

Proteins in both: 79268

GO terms by aspect (before filtering):
  C (Cellular Component): 2621
  F (Molecular Function): 6578
  P (Biological Process): 16781
  Total: 25980

=== Filtering to Allowed Terms (IA.tsv) ===
Allowed GO terms in IA.tsv: 40122

GO terms by aspect (after filtering to allowed terms):
  C (Cellular Component): 2621
  F (Molecular Function): 6578
  P (Biological Process): 16781
  Total: 25980

✅ Filtered to allowed terms - model will focus on terms that are actually scored!


In [27]:
# Step 3: Create target matrices for each aspect
print("=== Creating Target Matrices by Aspect ===")

# Filter embeddings to common proteins
esm2_id_to_idx = {pid: idx for idx, pid in enumerate(esm2_train_ids)}
common_indices = [esm2_id_to_idx[pid] for pid in common_proteins if pid in esm2_id_to_idx]
esm2_train_emb_filtered = esm2_train_emb[common_indices]

protein_to_idx = {pid: idx for idx, pid in enumerate(common_proteins)}
train_terms_filtered['protein_idx'] = train_terms_filtered['EntryID'].map(protein_to_idx)

# Create target matrices for each aspect
def create_aspect_target_matrix(go_terms, aspect_name, num_proteins):
    aspect_terms = train_terms_filtered[train_terms_filtered['term'].isin(go_terms)]
    
    target_df = aspect_terms.pivot_table(
        index='protein_idx',
        columns='term',
        values='EntryID',
        aggfunc='count',
        fill_value=0
    )
    
    # Ensure all proteins are present (even if they have no labels for this aspect)
    all_protein_indices = pd.Index(range(num_proteins), name='protein_idx')
    target_df = target_df.reindex(all_protein_indices, fill_value=0)
    
    # Ensure all GO terms are present (some might have no positives)
    for term in go_terms:
        if term not in target_df.columns:
            target_df[term] = 0
    
    # Reorder columns to match go_terms order
    target_df = target_df.reindex(columns=go_terms, fill_value=0)
    
    target_matrix = sp.csr_matrix(target_df.values.astype(np.float32))
    
    density = target_matrix.nnz / (target_matrix.shape[0] * target_matrix.shape[1])
    print(f"  {aspect_name}: {target_matrix.shape}, density: {density:.4%}, avg terms/protein: {target_matrix.sum(axis=1).mean():.2f}")
    return target_matrix

num_proteins = len(common_proteins)
y_train_C = create_aspect_target_matrix(go_terms_C, 'C', num_proteins)
y_train_F = create_aspect_target_matrix(go_terms_F, 'F', num_proteins)
y_train_P = create_aspect_target_matrix(go_terms_P, 'P', num_proteins)

print(f"\n✅ Target matrices created for all aspects")


=== Creating Target Matrices by Aspect ===
  C: (79268, 2621), density: 0.0747%, avg terms/protein: 1.96
  F: (79268, 6578), density: 0.0242%, avg terms/protein: 1.59
  P: (79268, 16781), density: 0.0185%, avg terms/protein: 3.11

✅ Target matrices created for all aspects


In [28]:
# Step 4: Split data into train/validation
print("=== Splitting Data ===")

train_idx, val_idx = train_test_split(
    np.arange(len(common_proteins)),
    test_size=0.2,
    random_state=42
)

X_train = esm2_train_emb_filtered[train_idx]
X_val = esm2_train_emb_filtered[val_idx]

y_train_C_split = y_train_C[train_idx]
y_val_C_split = y_train_C[val_idx]

y_train_F_split = y_train_F[train_idx]
y_val_F_split = y_train_F[val_idx]

y_train_P_split = y_train_P[train_idx]
y_val_P_split = y_train_P[val_idx]

print(f"Train: {X_train.shape[0]} proteins")
print(f"Validation: {X_val.shape[0]} proteins")


=== Splitting Data ===
Train: 63414 proteins
Validation: 15854 proteins


In [29]:
# Step 5: Build Multi-Aspect Models with MLP Architecture and Focal Loss
print("=== Building Multi-Aspect Models (MLP + Focal Loss) ===")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

class ProteinClassifier(nn.Module):
    """MLP classifier for one aspect (with hidden layer)"""
    def __init__(self, input_dim, output_dim, hidden_dim=512):
        super(ProteinClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

class FocalLoss(nn.Module):
    """Focal Loss for handling class imbalance
    
    FL(p_t) = -alpha * (1 - p_t)^gamma * log(p_t)
    
    Where:
    - p_t is predicted probability for true class
    - alpha: weighting factor (default 0.25)
    - gamma: focusing parameter (default 2.0, higher = more focus on hard examples)
    """
    def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
        
    def forward(self, logits, targets):
        # Convert logits to probabilities
        probs = torch.sigmoid(logits)
        
        # Calculate p_t (probability of true class)
        p_t = probs * targets + (1 - probs) * (1 - targets)
        
        # Calculate focal weight: (1 - p_t)^gamma
        focal_weight = (1 - p_t) ** self.gamma
        
        # Calculate BCE component
        bce = F.binary_cross_entropy_with_logits(logits, targets, reduction='none')
        
        # Apply alpha weighting (alpha for positive class, 1-alpha for negative)
        alpha_t = self.alpha * targets + (1 - self.alpha) * (1 - targets)
        
        # Focal loss
        focal_loss = alpha_t * focal_weight * bce
        
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

# Create separate MLP models for each aspect
hidden_dim = 512  # Hidden layer size
model_C = ProteinClassifier(X_train.shape[1], len(go_terms_C), hidden_dim=hidden_dim).to(device)
model_F = ProteinClassifier(X_train.shape[1], len(go_terms_F), hidden_dim=hidden_dim).to(device)
model_P = ProteinClassifier(X_train.shape[1], len(go_terms_P), hidden_dim=hidden_dim).to(device)

# Count parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\nModel C: {X_train.shape[1]} -> {hidden_dim} -> {len(go_terms_C)} GO terms")
print(f"  Parameters: {count_parameters(model_C):,}")
print(f"Model F: {X_train.shape[1]} -> {hidden_dim} -> {len(go_terms_F)} GO terms")
print(f"  Parameters: {count_parameters(model_F):,}")
print(f"Model P: {X_train.shape[1]} -> {hidden_dim} -> {len(go_terms_P)} GO terms")
print(f"  Parameters: {count_parameters(model_P):,}")

# Create Focal Loss for each aspect
# Focal Loss parameters (TUNE THESE if loss is too small or not decreasing):
# - alpha: weighting factor (0.25 = focus on positive class, try 0.5 if loss too small)
# - gamma: focusing parameter (2.0 = moderate, try 3.0-4.0 if loss too small, 1.0 if too large)
# - scale: loss scaling factor (1.0 = normal, try 10-100 if loss too small to provide gradients)
# PER-ASPECT Focal Loss parameters
# P aspect is weakest (0.10 Fmax), so we give it more aggressive settings
# C and F aspects (already performing well ~0.34-0.40)
focal_alpha_C = 0.5
focal_gamma_C = 4.0
focal_scale_C = 10.0

focal_alpha_F = 0.5
focal_gamma_F = 4.0
focal_scale_F = 10.0

# P aspect (weakest, needs more help)
focal_alpha_P = 0.5  # Same alpha
focal_gamma_P = 5.0   # Higher gamma = even more focus on hard examples
focal_scale_P = 15.0  # Higher scale = stronger gradients

# Wrap FocalLoss with scaling
class ScaledFocalLoss(nn.Module):
    def __init__(self, alpha, gamma, scale=1.0):
        super().__init__()
        self.focal_loss = FocalLoss(alpha=alpha, gamma=gamma, reduction='mean')
        self.scale = scale
    
    def forward(self, logits, targets):
        return self.focal_loss(logits, targets) * self.scale

criterion_C = ScaledFocalLoss(alpha=focal_alpha_C, gamma=focal_gamma_C, scale=focal_scale_C)
criterion_F = ScaledFocalLoss(alpha=focal_alpha_F, gamma=focal_gamma_F, scale=focal_scale_F)
criterion_P = ScaledFocalLoss(alpha=focal_alpha_P, gamma=focal_gamma_P, scale=focal_scale_P)

print(f"\n✅ Using PER-ASPECT FOCAL LOSS:")
print(f"   C: alpha={focal_alpha_C}, gamma={focal_gamma_C}, scale={focal_scale_C}")
print(f"   F: alpha={focal_alpha_F}, gamma={focal_gamma_F}, scale={focal_scale_F}")
print(f"   P: alpha={focal_alpha_P}, gamma={focal_gamma_P}, scale={focal_scale_P} (more aggressive for weak P aspect)")
print("   Tuning guide:")
print("   - Loss too small (<0.001) and not decreasing? → Increase gamma or scale")
print("   - Loss too large (>0.1)? → Decrease gamma or scale")
print("   - Loss decreasing but Fmax not improving? → Try more epochs")

# Optimizers with slightly lower learning rate (MLP can be more sensitive)
learning_rate = 0.0005  # TUNE: try 0.001 if loss not decreasing, 0.0001 if unstable
optimizer_C = torch.optim.Adam(model_C.parameters(), lr=learning_rate)
optimizer_F = torch.optim.Adam(model_F.parameters(), lr=learning_rate)
optimizer_P = torch.optim.Adam(model_P.parameters(), lr=learning_rate)

print(f"\n✅ MLP models created with Focal Loss")
print(f"   Learning rate: {learning_rate}")


=== Building Multi-Aspect Models (MLP + Focal Loss) ===
Using device: cpu

Model C: 1280 -> 512 -> 2621 GO terms
  Parameters: 2,000,445
Model F: 1280 -> 512 -> 6578 GO terms
  Parameters: 4,030,386
Model P: 1280 -> 512 -> 16781 GO terms
  Parameters: 9,264,525

✅ Using PER-ASPECT FOCAL LOSS:
   C: alpha=0.5, gamma=4.0, scale=10.0
   F: alpha=0.5, gamma=4.0, scale=10.0
   P: alpha=0.5, gamma=5.0, scale=15.0 (more aggressive for weak P aspect)
   Tuning guide:
   - Loss too small (<0.001) and not decreasing? → Increase gamma or scale
   - Loss too large (>0.1)? → Decrease gamma or scale
   - Loss decreasing but Fmax not improving? → Try more epochs

✅ MLP models created with Focal Loss
   Learning rate: 0.0005


In [30]:
# Step 6: Create DataLoaders for each aspect
print("=== Creating DataLoaders ===")

class ProteinDataset(Dataset):
    def __init__(self, embeddings, targets):
        self.embeddings = embeddings
        self.targets = targets
    
    def __len__(self):
        return len(self.embeddings)
    
    def __getitem__(self, idx):
        emb = torch.tensor(self.embeddings[idx], dtype=torch.float32)
        target = torch.tensor(self.targets[idx].toarray().flatten(), dtype=torch.float32)
        return emb, target

batch_size = 32

train_dataset_C = ProteinDataset(X_train, y_train_C_split)
train_dataset_F = ProteinDataset(X_train, y_train_F_split)
train_dataset_P = ProteinDataset(X_train, y_train_P_split)

val_dataset_C = ProteinDataset(X_val, y_val_C_split)
val_dataset_F = ProteinDataset(X_val, y_val_F_split)
val_dataset_P = ProteinDataset(X_val, y_val_P_split)

train_loader_C = DataLoader(train_dataset_C, batch_size=batch_size, shuffle=True, num_workers=0)
train_loader_F = DataLoader(train_dataset_F, batch_size=batch_size, shuffle=True, num_workers=0)
train_loader_P = DataLoader(train_dataset_P, batch_size=batch_size, shuffle=True, num_workers=0)

val_loader_C = DataLoader(val_dataset_C, batch_size=batch_size, shuffle=False, num_workers=0)
val_loader_F = DataLoader(val_dataset_F, batch_size=batch_size, shuffle=False, num_workers=0)
val_loader_P = DataLoader(val_dataset_P, batch_size=batch_size, shuffle=False, num_workers=0)

print(f"Train batches - C: {len(train_loader_C)}, F: {len(train_loader_F)}, P: {len(train_loader_P)}")
print(f"Val batches - C: {len(val_loader_C)}, F: {len(val_loader_F)}, P: {len(val_loader_P)}")


=== Creating DataLoaders ===
Train batches - C: 1982, F: 1982, P: 1982
Val batches - C: 496, F: 496, P: 496


In [31]:
# Step 7: Training Loop for Multi-Aspect Models
print("=== Training Multi-Aspect Models ===")

def train_aspect_model(model, train_loader, val_loader, criterion, optimizer, aspect_name, num_epochs=3):
    print(f"\nTraining {aspect_name} model...")
    print(f"  Training for {num_epochs} epochs")
    print(f"  Watch for: Loss should decrease, train/val should be similar")
    
    # Get initial prediction stats (before training)
    model.eval()
    with torch.no_grad():
        sample_emb, sample_target = next(iter(val_loader))
        sample_emb = sample_emb.to(device)
        initial_outputs = model(sample_emb)
        initial_probs = torch.sigmoid(initial_outputs)
        initial_mean_prob = initial_probs.mean().item()
        initial_max_prob = initial_probs.max().item()
        initial_loss = criterion(initial_outputs, sample_target.to(device)).item()
    
    print(f"  Initial: Loss={initial_loss:.6f}, Mean prob={initial_mean_prob:.4f}, Max prob={initial_max_prob:.4f}")
    
    # Track loss history
    train_losses = []
    val_losses = []
    
    for epoch in range(num_epochs):
        # Training
        model.train()
        train_loss = 0.0
        for batch_idx, (emb_batch, target_batch) in enumerate(train_loader):
            emb_batch = emb_batch.to(device)
            target_batch = target_batch.to(device)
            
            outputs = model(emb_batch)
            loss = criterion(outputs, target_batch)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            
            if (batch_idx + 1) % 500 == 0:
                print(f"  {aspect_name} Epoch {epoch+1}/{num_epochs}, Batch {batch_idx+1}/{len(train_loader)}, Loss: {loss.item():.4f}")
            
            del outputs, loss
            if (batch_idx + 1) % 200 == 0:
                gc.collect()
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
        
        avg_train_loss = train_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        
        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for emb_batch, target_batch in val_loader:
                emb_batch = emb_batch.to(device)
                target_batch = target_batch.to(device)
                outputs = model(emb_batch)
                val_loss += criterion(outputs, target_batch).item()
                del outputs
        
        avg_val_loss = val_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        
        # Check if predictions are changing
        with torch.no_grad():
            final_outputs = model(sample_emb)
            final_probs = torch.sigmoid(final_outputs)
            final_mean_prob = final_probs.mean().item()
            final_max_prob = final_probs.max().item()
            final_loss = criterion(final_outputs, sample_target.to(device)).item()
        
        # Loss change indicator
        loss_change = ""
        if epoch > 0:
            prev_train = train_losses[-2]
            prev_val = val_losses[-2]
            train_change = ((avg_train_loss - prev_train) / prev_train) * 100
            val_change = ((avg_val_loss - prev_val) / prev_val) * 100
            loss_change = f" (Train: {train_change:+.1f}%, Val: {val_change:+.1f}%)"
        
        print(f"  {aspect_name} Epoch {epoch+1}/{num_epochs} - Train Loss: {avg_train_loss:.6f}, Val Loss: {avg_val_loss:.6f}{loss_change}")
        print(f"    Sample loss: {initial_loss:.6f} → {final_loss:.6f}, Mean prob: {initial_mean_prob:.4f} → {final_mean_prob:.4f}")
        
        # Warning if loss not decreasing
        if epoch > 0 and avg_train_loss >= train_losses[-2] * 0.99:  # Less than 1% decrease
            print(f"    ⚠️  Loss barely decreasing - consider: more epochs, higher gamma/scale, or higher LR")
        
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    
    # Final summary
    total_decrease = ((train_losses[-1] - train_losses[0]) / train_losses[0]) * 100
    print(f"  ✅ {aspect_name} complete! Loss decreased by {total_decrease:.1f}% over {num_epochs} epochs")
    if total_decrease > -5:  # Less than 5% decrease
        print(f"     ⚠️  Small decrease - model may need: more epochs, better focal params, or higher LR")

# TUNE: Number of epochs PER ASPECT
# - C and F are already good (~0.34-0.40 Fmax), so moderate epochs
# - P is weak (0.10 Fmax), so give it MORE epochs to learn
# - Loss still decreasing at end? → Increase epochs
# - Loss plateaued early? → Don't increase epochs, tune focal loss instead

num_epochs_C = 5   # C aspect (already good)
num_epochs_F = 5   # F aspect (already good)
num_epochs_P = 10  # P aspect (weakest, needs more training)

print(f"\n{'='*60}")
print(f"TRAINING CONFIGURATION (PER-ASPECT)")
print(f"{'='*60}")
print(f"Epochs - C: {num_epochs_C}, F: {num_epochs_F}, P: {num_epochs_P} (P gets more!)")
print(f"Focal Loss C: alpha={focal_alpha_C}, gamma={focal_gamma_C}, scale={focal_scale_C}")
print(f"Focal Loss F: alpha={focal_alpha_F}, gamma={focal_gamma_F}, scale={focal_scale_F}")
print(f"Focal Loss P: alpha={focal_alpha_P}, gamma={focal_gamma_P}, scale={focal_scale_P}")
print(f"Learning Rate: {learning_rate}")
print(f"{'='*60}\n")

# Train each aspect model with different epochs
train_aspect_model(model_C, train_loader_C, val_loader_C, criterion_C, optimizer_C, 'C', num_epochs_C)
train_aspect_model(model_F, train_loader_F, val_loader_F, criterion_F, optimizer_F, 'F', num_epochs_F)
train_aspect_model(model_P, train_loader_P, val_loader_P, criterion_P, optimizer_P, 'P', num_epochs_P)

print("\n✅ All models trained!")
print("\n💡 Next step: Run evaluation cell to check Fmax. If Fmax is lower than expected:")
print("   - Loss was too small/plateaued? → Increase focal_gamma (2.0→3.0) or focal_scale (1.0→10)")
print("   - Loss was decreasing? → Try more epochs (3→5)")
print("   - Loss was unstable? → Decrease learning_rate (0.0005→0.0001)")


=== Training Multi-Aspect Models ===

TRAINING CONFIGURATION (PER-ASPECT)
Epochs - C: 5, F: 5, P: 10 (P gets more!)
Focal Loss C: alpha=0.5, gamma=4.0, scale=10.0
Focal Loss F: alpha=0.5, gamma=4.0, scale=10.0
Focal Loss P: alpha=0.5, gamma=5.0, scale=15.0
Learning Rate: 0.0005


Training C model...
  Training for 5 epochs
  Watch for: Loss should decrease, train/val should be similar
  Initial: Loss=0.218728, Mean prob=0.5000, Max prob=0.5562
  C Epoch 1/5, Batch 500/1982, Loss: 0.0020
  C Epoch 1/5, Batch 1000/1982, Loss: 0.0014
  C Epoch 1/5, Batch 1500/1982, Loss: 0.0018
  C Epoch 1/5 - Train Loss: 0.002884, Val Loss: 0.001678
    Sample loss: 0.218728 → 0.001692, Mean prob: 0.5000 → 0.0967
  C Epoch 2/5, Batch 500/1982, Loss: 0.0016
  C Epoch 2/5, Batch 1000/1982, Loss: 0.0018
  C Epoch 2/5, Batch 1500/1982, Loss: 0.0015
  C Epoch 2/5 - Train Loss: 0.001534, Val Loss: 0.001546 (Train: -46.8%, Val: -7.9%)
    Sample loss: 0.218728 → 0.001662, Mean prob: 0.5000 → 0.0818
  C Epoch 3/

In [32]:
# Step 8: Evaluate on validation set using Fmax (per aspect) - Top-50
print("=== Evaluating on Validation Set (Fmax) ===\n")

def apply_top_k_filter(predictions, top_k):
    """Apply top-k filtering to predictions (keep only top-k per protein)"""
    filtered_predictions = np.zeros_like(predictions)
    for i in range(predictions.shape[0]):
        top_k_indices = np.argpartition(predictions[i], -top_k)[-top_k:]
        filtered_predictions[i, top_k_indices] = predictions[i, top_k_indices]
    return filtered_predictions

def calculate_fmax(predictions, targets, thresholds=np.linspace(0.01, 0.99, 50)):
    """Calculate Fmax for predictions vs targets"""
    total_positives = np.sum(targets)
    f1_scores = []
    
    for threshold in thresholds:
        pred_binary = (predictions >= threshold).astype(bool)
        tp = np.sum(pred_binary & targets.astype(bool))
        fp = np.sum(pred_binary & (~targets.astype(bool)))
        fn = total_positives - tp
        
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0
        f1_scores.append(f1)
    
    fmax = max(f1_scores)
    best_threshold_idx = np.argmax(f1_scores)
    best_threshold = thresholds[best_threshold_idx]
    
    # Calculate precision/recall at best threshold
    pred_binary = (predictions >= best_threshold).astype(float)
    tp = np.sum(pred_binary * targets)
    fp = np.sum(pred_binary * (1 - targets))
    fn = np.sum((1 - pred_binary) * targets)
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    
    return fmax, precision, recall

# Get predictions for each aspect
def get_predictions(model, val_loader, device):
    model.eval()
    predictions = []
    with torch.no_grad():
        for emb_batch, _ in val_loader:
            emb_batch = emb_batch.to(device)
            outputs = model(emb_batch)
            probs = torch.sigmoid(outputs).cpu().numpy()
            predictions.append(probs)
    return np.vstack(predictions)

print("Loading predictions...")
val_preds_C = get_predictions(model_C, val_loader_C, device)
val_preds_F = get_predictions(model_F, val_loader_F, device)
val_preds_P = get_predictions(model_P, val_loader_P, device)

val_targets_C = np.vstack([target.numpy() for _, target in val_loader_C])
val_targets_F = np.vstack([target.numpy() for _, target in val_loader_F])
val_targets_P = np.vstack([target.numpy() for _, target in val_loader_P])

# Apply top-50 filtering (C:20, F:20, P:10)
print("Applying top-50 filtering (C:20, F:20, P:10)...")
val_preds_C = apply_top_k_filter(val_preds_C, top_k=20)
val_preds_F = apply_top_k_filter(val_preds_F, top_k=20)
val_preds_P = apply_top_k_filter(val_preds_P, top_k=10)

# Calculate Fmax for each aspect
fmax_C, prec_C, rec_C = calculate_fmax(val_preds_C, val_targets_C)
fmax_F, prec_F, rec_F = calculate_fmax(val_preds_F, val_targets_F)
fmax_P, prec_P, rec_P = calculate_fmax(val_preds_P, val_targets_P)

print(f"\nC (Cellular Component):")
print(f"  Fmax: {fmax_C:.4f}, Precision: {prec_C:.4f}, Recall: {rec_C:.4f}")

print(f"\nF (Molecular Function):")
print(f"  Fmax: {fmax_F:.4f}, Precision: {prec_F:.4f}, Recall: {rec_F:.4f}")

print(f"\nP (Biological Process):")
print(f"  Fmax: {fmax_P:.4f}, Precision: {prec_P:.4f}, Recall: {rec_P:.4f}")

avg_fmax = np.mean([fmax_C, fmax_F, fmax_P])
print(f"\n{'='*50}")
print(f"Average Fmax (CAFA metric): {avg_fmax:.4f}")
print(f"{'='*50}")

validation_fmax = avg_fmax


=== Evaluating on Validation Set (Fmax) ===

Loading predictions...
Applying top-50 filtering (C:20, F:20, P:10)...

C (Cellular Component):
  Fmax: 0.3425, Precision: 0.3206, Recall: 0.3675

F (Molecular Function):
  Fmax: 0.4026, Precision: 0.4041, Recall: 0.4010

P (Biological Process):
  Fmax: 0.1256, Precision: 0.1356, Recall: 0.1170

Average Fmax (CAFA metric): 0.2902


In [33]:
# Step 9: Make test predictions and create submission
print("=== Making Test Predictions & Creating Submission ===")

print(f"Test embeddings: {esm2_test_emb.shape}")
print(f"Test IDs: {len(esm2_test_ids)}")

output_file = 'submission.tsv'

# Prediction strategy: 'top_k' or 'threshold'
PREDICTION_STRATEGY = 'top_k'  # Change to 'threshold' for threshold-based

if PREDICTION_STRATEGY == 'top_k':
    # Top-k per aspect (best from evaluation: 50 total)
    top_k_per_aspect = {'C': 20, 'F': 20, 'P': 10}  # Total = 50
    print(f"Using TOP-K strategy: {top_k_per_aspect}")
else:
    # Threshold-based: output all terms above threshold
    prediction_threshold = 0.001  # Only terms with confidence > 0.001
    print(f"Using THRESHOLD strategy: p > {prediction_threshold}")

# Process in chunks to avoid memory issues
chunk_size = 5000
num_chunks = (len(esm2_test_emb) + chunk_size - 1) // chunk_size

print(f"Processing in {num_chunks} chunks of {chunk_size} proteins each...")

total_predictions = 0

with open(output_file, 'w') as f:
    for chunk_idx in range(num_chunks):
        start_idx = chunk_idx * chunk_size
        end_idx = min((chunk_idx + 1) * chunk_size, len(esm2_test_emb))
        chunk_proteins = end_idx - start_idx
        
        print(f"\nChunk {chunk_idx + 1}/{num_chunks}: Processing proteins {start_idx} to {end_idx}...")
        
        # Get predictions for this chunk from each model
        chunk_emb = torch.tensor(esm2_test_emb[start_idx:end_idx], dtype=torch.float32).to(device)
        
        model_C.eval()
        model_F.eval()
        model_P.eval()
        
        with torch.no_grad():
            probs_C = torch.sigmoid(model_C(chunk_emb)).cpu().numpy()
            probs_F = torch.sigmoid(model_F(chunk_emb)).cpu().numpy()
            probs_P = torch.sigmoid(model_P(chunk_emb)).cpu().numpy()
        
        # Write predictions for each protein
        for i, protein_idx in enumerate(range(start_idx, end_idx)):
            protein_id = esm2_test_ids[protein_idx]
            all_predictions = []
            
            if PREDICTION_STRATEGY == 'top_k':
                # Top-k per aspect
                # C predictions
                top_k_C = top_k_per_aspect['C']
                top_k_indices_C = np.argpartition(probs_C[i], -top_k_C)[-top_k_C:]
                top_k_indices_C = top_k_indices_C[np.argsort(probs_C[i][top_k_indices_C])][::-1]
                for idx in top_k_indices_C:
                    all_predictions.append((go_terms_C[idx], float(probs_C[i][idx])))
                
                # F predictions
                top_k_F = top_k_per_aspect['F']
                top_k_indices_F = np.argpartition(probs_F[i], -top_k_F)[-top_k_F:]
                top_k_indices_F = top_k_indices_F[np.argsort(probs_F[i][top_k_indices_F])][::-1]
                for idx in top_k_indices_F:
                    all_predictions.append((go_terms_F[idx], float(probs_F[i][idx])))
                
                # P predictions
                top_k_P = top_k_per_aspect['P']
                top_k_indices_P = np.argpartition(probs_P[i], -top_k_P)[-top_k_P:]
                top_k_indices_P = top_k_indices_P[np.argsort(probs_P[i][top_k_indices_P])][::-1]
                for idx in top_k_indices_P:
                    all_predictions.append((go_terms_P[idx], float(probs_P[i][idx])))
            
            else:
                # Threshold-based: output all terms above threshold
                # C predictions
                above_threshold_C = probs_C[i] > prediction_threshold
                for idx in np.where(above_threshold_C)[0]:
                    all_predictions.append((go_terms_C[idx], float(probs_C[i][idx])))
                
                # F predictions
                above_threshold_F = probs_F[i] > prediction_threshold
                for idx in np.where(above_threshold_F)[0]:
                    all_predictions.append((go_terms_F[idx], float(probs_F[i][idx])))
                
                # P predictions
                above_threshold_P = probs_P[i] > prediction_threshold
                for idx in np.where(above_threshold_P)[0]:
                    all_predictions.append((go_terms_P[idx], float(probs_P[i][idx])))
            
            # Write to file (sorted by confidence descending)
            all_predictions.sort(key=lambda x: x[1], reverse=True)
            for go_term, confidence in all_predictions:
                f.write(f"{protein_id}\t{go_term}\t{confidence}\n")
                total_predictions += 1
        
        # Free memory
        del chunk_emb, probs_C, probs_F, probs_P
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        
        print(f"  ✅ Chunk {chunk_idx + 1} complete!")

print(f"\n✅ Submission created: {output_file}")
print(f"   Total predictions: {total_predictions}")
print(f"   File size: {os.path.getsize(output_file) / (1024*1024):.2f} MB")


=== Making Test Predictions & Creating Submission ===
Test embeddings: (141864, 1280)
Test IDs: 141864
Using TOP-K strategy: {'C': 20, 'F': 20, 'P': 10}
Processing in 29 chunks of 5000 proteins each...

Chunk 1/29: Processing proteins 0 to 5000...
  ✅ Chunk 1 complete!

Chunk 2/29: Processing proteins 5000 to 10000...
  ✅ Chunk 2 complete!

Chunk 3/29: Processing proteins 10000 to 15000...
  ✅ Chunk 3 complete!

Chunk 4/29: Processing proteins 15000 to 20000...
  ✅ Chunk 4 complete!

Chunk 5/29: Processing proteins 20000 to 25000...
  ✅ Chunk 5 complete!

Chunk 6/29: Processing proteins 25000 to 30000...
  ✅ Chunk 6 complete!

Chunk 7/29: Processing proteins 30000 to 35000...
  ✅ Chunk 7 complete!

Chunk 8/29: Processing proteins 35000 to 40000...
  ✅ Chunk 8 complete!

Chunk 9/29: Processing proteins 40000 to 45000...
  ✅ Chunk 9 complete!

Chunk 10/29: Processing proteins 45000 to 50000...
  ✅ Chunk 10 complete!

Chunk 11/29: Processing proteins 50000 to 55000...
  ✅ Chunk 11 complet